In [85]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [86]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [8,16,32,64,128,256]

In [87]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [88]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)


In [89]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()

In [90]:
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()

In [91]:
df_ma.dropna(inplace=True)

In [92]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_8,MA_16,MA_32,MA_64,MA_128,MA_256
255,2022-06-22T16:00:00.000000000Z,1.05800,1.05896,1.05702,1.05750,1.054919,1.052567,1.053419,1.052957,1.050319,1.053098
256,2022-06-22T17:00:00.000000000Z,1.05750,1.05833,1.05690,1.05786,1.055661,1.053010,1.053452,1.053090,1.050385,1.053056
257,2022-06-22T18:00:00.000000000Z,1.05786,1.05846,1.05655,1.05670,1.056246,1.053368,1.053498,1.053166,1.050451,1.053007
258,2022-06-22T19:00:00.000000000Z,1.05671,1.05748,1.05652,1.05670,1.056939,1.053719,1.053551,1.053226,1.050536,1.052962
259,2022-06-22T20:00:00.000000000Z,1.05671,1.05692,1.05634,1.05688,1.057414,1.054170,1.053604,1.053297,1.050656,1.052919


In [93]:
df_plot = df_ma.iloc[-200:].copy()

In [94]:
df_plot.shape


(200, 11)

In [95]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
# This plots the moving average indicator
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()